In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


In [2]:
#-- Add REAL WORLD configuration and camera
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

# Change object color in RealWorld to red (should add)
obj1 = RealWorld.frame("obj1")
obj1.setColor([1.,.0,.0])

S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

In [3]:
#the focal length
f = 0.895
f = f * 360.
#the relative pose of the camera
# pcl.setRelativePose('d(-90 0 0 1) t(-.08 .205 .115) d(26 1 0 0) d(-1 0 1 0) d(6 0 0 1) ')
fxfypxpy = [f, f, 320., 180.]

**Ex 2**

In [4]:
points = []
tau = .005

# initialize the first frame in the video stream
firstFrame = None

for t in range(300):
    time.sleep(0.1)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        
        # Convert depth to point cloud. (n*3) shape
        points = S.depthData2pointCloud(depth, fxfypxpy)
        
        # Convert BGR to RGB. Then to only detect changes, only gray scale enough
        frame = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
        frame_g = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # Blurr, so not to detect minute differences and noise in each consecutive frame
        # Gaussian blurr 21 X 21
        # frame_blurr = cv.GaussianBlur(frame_g, (21, 21), 0)

        # If the first frame is None, initialize it
        if firstFrame is None:
            firstFrame = frame_g
            continue

        # compute the absolute difference between the current frame and
        # first frame
        frame_diff = cv.absdiff(firstFrame, frame_g)

        # If difference is larger than 25 then consider the change
        # retval, dst=cv.threshold(src, thresh, maxval, type[, dst])
        _, frame_thresh = cv.threshold(frame_diff, 30, 255, cv.THRESH_BINARY)

        # Dilate the thresholded image to fill in holes, then find contours on thresholded image
        frame_dil = cv.dilate(frame_thresh, None, iterations=2)

        # Find contours
        frame_contours, _ = cv.findContours(frame_dil, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        # Draw contours for all objects detected
        output_frame = frame.copy()
        for contour in frame_contours:
            # If contour area too small ignore it
            # if cv.contourArea(contour) < 200:
                #continue

            # Draw contour
            # image =cv.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
            cv.drawContours(output_frame, contour, -1, (0, 255, 0), 3)

        # Display the resulting frame
        if len(frame)>0:cv.imshow('Input_Frame',frame)
        if len(frame_diff)>0:cv.imshow('Diff_Frame',frame_diff)
        #if len(frame_thresh)>0:cv.imshow('Threshold_Frame',frame_thresh)
        if len(output_frame)>0:cv.imshow('Output_Frame',output_frame)

        # Setting the object in the config space, as point cloud
        cameraFrame.setPointCloud(points, rgb)
        
        # updating the display
        V.recopyMeshes(C)
        V.setConfiguration(C)
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)

**Ex 3**

In [4]:
points = []
tau = .005

# initialize the first frame in the video stream
InitialDepth = None
maxdepth = 0

for t in range(300):
    time.sleep(0.1)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        
        # Convert BGR to RGB.
        frame = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
        
        # Convert depth to point cloud. (n*3) shape
        points = S.depthData2pointCloud(depth, fxfypxpy)

        # If the first frame is None, initialize it
        if InitialDepth is None:
            maxdepth = np.amax(depth)
            InitialDepth = (np.floor((depth.astype(np.float64) / maxdepth) * 255)).astype(np.uint8)
            continue

        # Normalise ans scale depth array
        depth_NS = (np.floor((depth.astype(np.float64) / maxdepth) * 255)).astype(np.uint8)
        
        # compute the absolute difference between the current frame and
        # first frame
        depth_diff = InitialDepth - depth_NS
        # Set my output imgage pixel value to zero everywhere except my mask
        depth_diff = depth_diff.clip(min=0)
        #depth_diff = cv.absdiff(InitialDepth, depth_NS)

        # If difference is larger than 25 then consider the change
        # retval, dst=cv.threshold(src, thresh, maxval, type[, dst])
        _, depth_thresh = cv.threshold(depth_diff, 25, 255, cv.THRESH_BINARY)

        # Find contours
        depth_contours, _ = cv.findContours(depth_thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
        
        # Draw contours for all objects detected
        output_frame = frame.copy()
        for contour in depth_contours:
            # If contour area too small ignore it
            # if cv.contourArea(contour) < 200:
                #continue

            # Draw contour
            # image =cv.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
            cv.drawContours(output_frame, contour, -1, (0, 255, 0), 3)

        # Display the resulting frame
        if len(frame)>0:cv.imshow('Input_Frame',frame)
        if len(depth)>0: cv.imshow('Depth_Frame', 0.5* depth)
        if len(depth_diff)>0:cv.imshow('Diff_Frame',depth_diff)
        if len(depth_thresh)>0:cv.imshow('Threshold_Frame',depth_thresh)
        if len(output_frame)>0:cv.imshow('Output_Frame',output_frame)

        # Setting the object in the config space, as point cloud
        cameraFrame.setPointCloud(points, rgb)
        
        # updating the display
        V.recopyMeshes(C)
        V.setConfiguration(C)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)

In [5]:
cv.destroyAllWindows


<function destroyAllWindows>

In [6]:
def scale_normalise(arr, maxm):
    return arr = np.floor((arr / maxm) * 255)

SyntaxError: invalid syntax (<ipython-input-6-5e80401329f2>, line 2)

In [ ]:
# if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
# if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

## example to grap images from a webcam
(but this only gives you an rgb, not depth)
hit 'q' in the window to stop

**Ex 1 - Static Image**

In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


In [ ]:
# Ex 1 - For static image
# Load an color image as HSV

in_img = cv.imread('images/all24.jpg')
height = np.size(in_img, 0)
width  = np.size(in_img, 1)

print ("Width = ", width, "Height = ", height )

# converting from BGR to HSV color space
hsv_img = cv.cvtColor(in_img,cv.COLOR_BGR2HSV)

# Ex 1.1 - Filter the color of the image to find all pixels that are redish.

# Threshold the HSV image, keep only the red pixels
red_lowerth = np.array([0,100,90])
red_upperth = np.array([10,255,255])
red_mask1 = cv.inRange(hsv_img, red_lowerth, red_upperth)
 
# Range for upper range
red_lowerth = np.array([160,100,90])
red_upperth = np.array([180,255,255])
red_mask2 = cv.inRange(hsv_img,red_lowerth,red_upperth)

# Generating the final mask to detect red color and find all pixels that are 
red_mask = cv.addWeighted(red_mask1, 1, red_mask2, 1, 0)

# Ex 1.2 - Display the binary image that indicates red pixels.
# Set my output imgage pixel value to zero everywhere except my mask
output_img = in_img.copy()
output_img[np.where(red_mask==0)] = 0

cv.imshow("Red_Masked_Image",output_img)

# Ex 1.3 - Fit contours to this binary image, which returns the segments.

# Find contours for the objects detected in mask
# contours, hierarchy=cv.findContours(image, mode, method[, contours[, hierarchy[, offset]]])
contours, _ = cv.findContours(red_mask, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    
# Ex 1.4 - Display the contours by drawing them into the original RGB image
# Draw contours for all objects detected
output_img = in_img.copy()
for contour in contours:
    # If contour area too small ignore it
    # if abs(cv.contourArea(contour)) < 10000:
        # continue
    # image	=cv.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
    cv.drawContours(output_img, contour, -1, (0, 255, 0), 3)
    
cv.imshow("Contoured Objects", output_img)

cv.waitKey(0)
cv.destroyAllWindows()
    

Width =  800 Height =  800


**Ex 1 - Webcam**

In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


In [9]:
# Ex 1 - For webcam
cap = cv.VideoCapture(0)
while(True):
    
    # Capture frame-by-frame
    ret, frame = cap.read()

    # converting from BGR to HSV color space
    hsv_img = cv.cvtColor(frame,cv.COLOR_BGR2HSV)

    # Ex 1.1 - Filter the color of the image to find all pixels that are redish.
    # Threshold the HSV image, keep only the red pixels
    red_lowerth = np.array([0,105,70])
    red_upperth = np.array([10,255,255])
    red_mask1 = cv.inRange(hsv_img, red_lowerth, red_upperth)

    # Range for upper range
    red_lowerth = np.array([170,105,70])
    red_upperth = np.array([180,255,255])
    red_mask2 = cv.inRange(hsv_img,red_lowerth,red_upperth)

    # Generating the final mask to detect red color and find all pixels that are 
    red_mask = cv.addWeighted(red_mask1, 1, red_mask2, 1, 0)

    # Ex 1.2 - Display the binary image that indicates red pixels.
    # Set my output imgage pixel value to zero everywhere except my mask
    output_mask = frame.copy()
    output_mask[np.where(red_mask==0)] = 0

    # Ex 1.3 - Fit contours to this binary image, which returns the segments.
    # Find contours for the objects detected in mask
    # contours, hierarchy=cv.findContours(image, mode, method[, contours[, hierarchy[, offset]]])
    contours, _ = cv.findContours(red_mask, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    # Ex 1.4 - Display the contours by drawing them into the original RGB image
    # Draw contours for all objects detected
    output_frame = frame.copy()
    for contour in contours:
        # If contour area too small ignore it
        # if cv2.contourArea(c) < :
        #    continue
        # image =cv.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
        cv.drawContours(output_frame, contour, -1, (0, 255, 0), )
    
    # Display the resulting frame
    if len(frame)>0:cv.imshow('Input_Frame',frame)
    if len(output_mask)>0:cv.imshow('Output_Mask',output_mask)
    if len(output_frame)>0:cv.imshow('Output_Frame',output_frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

**Ex 2 - Over a video**

In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


In [3]:
# Ex 2 - 
# cap = cv.VideoCapture(0)
cap = cv.VideoCapture('images/car-overhead-1.avi')

# initialize the first frame in the video stream
firstFrame = None

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If the frame could not be grabbed, then we have reached the end of the video
    if frame is None:
        break
        
    # To only detect changes, gray scale enough
    frame_g = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Blurr, so not not detect minute differences and noise in each consecutive frame
    # Gaussian blurr 21 X 21
    frame_blurr = cv.GaussianBlur(frame_g, (21, 21), 0)
    
    # 2.1 Store the first image as background image. (Or average over several first images.)
    # If the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = frame_g
        continue
    
    # 2.2 For every new image filter those pixels, that are significantly different to the background.
    # Compute the absolute difference between the current frame and
    frame_diff = cv.absdiff(firstFrame, frame_g)
    
    # 2.6. Test the same pipeline, but first smoothing/blurring the image
    # If difference is larger than 25 then consider the change
    # retval, dst=cv.threshold(src, thresh, maxval, type[, dst])
    _, frame_thresh = cv.threshold(frame_diff, 25, 255, cv.THRESH_BINARY)
    
    # Dilate the thresholded image to fill in holes, then find contours on thresholded image
    frame_dil = cv.dilate(frame_thresh, None, iterations=2)
    
    # 2.4. Fit contours to this binary image, which returns the segments.
    # Find contours
    frame_contours, _ = cv.findContours(frame_dil, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    # 2.5. Display the contours by drawing them into the original RGB image
    # Draw contours for all objects detected
    output_frame = frame.copy()
    for contour in frame_contours:
        # If contour area too small ignore it
        if cv.contourArea(contour) < 200:
            continue
        
        # Draw contour
        # image =cv.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
        cv.drawContours(output_frame, contour, -1, (0, 255, 0), 3)
        
    # Display the resulting frame
    if len(frame)>0:cv.imshow('Input_Frame',frame)
        
    # 2.3. Display the binary image that indicates change pixels.
    if len(frame_diff)>0:cv.imshow('Diff_Frame',frame_diff)
        
    #if len(frame_thresh)>0:cv.imshow('Threshold_Frame',frame_thresh)
    if len(output_frame)>0:cv.imshow('Output_Frame',output_frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        # When everything done, release the capture
        cap.release()
        break

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

    